# Goal: Extract DC for all non-liq pts. If =0, then drop.

In [1]:
# Alex Chansky
# 3/14/21
# Purpose: Identifying and dropping non-liq cells near liq cells
import arcpy
from arcpy import env
import pandas as pd
import os
arcpy.CheckOutExtension("Spatial") # This checks out the license
from arcpy.sa import *
# If getting "Not Signed into Portal" message, should open up and log into ArcGIS Pro

In [36]:
import datetime

In [19]:
region_path = "C:\\Users\\achans01\\Downloads\\WatBods\\"
path = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\Liquefaction_inventories\\2021_liq_Alex\\"
temp_path = "C:\\Users\\achans01\\Downloads\\Arc"
eq_list = pd.read_csv("R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\Liquefaction_inventories\\2021_liq_Alex\\z_eq_names\\2021_eq_names.csv")
num_eq = eq_list.shape[0]
eq_list_df = pd.DataFrame(eq_list)

In [20]:
eq_list_df

,Name,Date,Magnitude,NewFolder,Projection,Unnamed: 5,ProjectionCode,Working,Liq,Geometry,Region
0,Achia,6/8/2008,6.5,Achaia,Greek_Grid,NaN,2100,1,1,PtsOnly,Greece
1,Aquila,4/6/2009,6.3,Aquila,RDN2008 TM33,NaN,6708,1,1,PtsOnly,Italy
2,Arequipa,6/23/2001,8.4,Arequipa,SIRGAS 2000 UTM Zone 18S,NaN,31978,1,1,PtsOnly,PeruChile
3,Baja California,4/4/2010,7.2,BajaCalifornia,Mexican Datum 1993 UTM Zone 11N,NaN,4484,1,1,PtsOnly,CaliforniaSouth
4,Bhuj,1/26/2001,7.0,Bhuj,Kalianpur 1975 India Zone IIa,NaN,24379,1,1,PolyOnly,Bhuj
5,Central Italy,8/24/2016,6.2,CentralItaly,RDN2008 TM33,NaN,6708,1,0,None,Italy
6,Cephalonia,1/26/2014,6.1,Cephalonia,Greek Grid,NaN,2100,1,1,PtsOnly,Greece
7,Chiba,12/17/1983,6.5,Chiba,JGD 2011 Zone 9,NaN,6677,1,1,Both,JapanNorth
8,Chi-Chi,9/21/1999,6.3,ChiChi,TWD 1997 Penghu,NaN,3825,1,1,Both,Taiwan
9,Chino Hills,7/29/2008,5.4,ChinoHills,CA state plane 5,NaN,26945,1,0,None,CaliforniaSouth


In [22]:
for x in range(9,num_eq):
    eq_name = eq_list['NewFolder'][x]
    region = eq_list['Region'][x]
    if eq_list['Working'][x]==1 and eq_list['Liq'][x]==1:
        
        points = path + eq_name + "\\fish_non_liq_point.shp"
        dc = region_path + "ASTER_DCs\\"+ region + "_ocean.tif"
        
        ExtractMultiValuesToPoints(points, [[dc,"DC"]], "NONE")
        
        # Create copy to drop DC = 0 points in case we want those back for any reason
        points_copy = path + eq_name + "\\fish_non_liq_point_copy.shp"
        arcpy.CopyFeatures_management(points, points_copy)
        
        aa = arcpy.SelectLayerByAttribute_management(points_copy, 'NEW_SELECTION', '"DC" = 0')
        arcpy.management.DeleteFeatures(aa)
    
    if eq_list['Working'][x]==1 and eq_list['Liq'][x]==0:
        
        points = path + eq_name + "\\non_liq_point.shp" # This is the only line changed from above seciton
                                                        # because NL event NL points named differently
        dc = region_path + "ASTER_DCs\\"+ region + "_ocean.tif"
        
        ExtractMultiValuesToPoints(points, [[dc,"DC"]], "NONE")
        
        # Create copy to drop DC = 0 points in case we want those back for any reason
        points_copy = path + eq_name + "\\fish_non_liq_point_copy.shp"
        arcpy.CopyFeatures_management(points, points_copy)
        
        aa = arcpy.SelectLayerByAttribute_management(points_copy, 'NEW_SELECTION', '"DC" = 0')
        arcpy.management.DeleteFeatures(aa)
      
    else:
        pass
    
    print("Successfully deleted DC=0 for " + eq_name)

Successfully deleted DC=0 for ChinoHills
Successfully deleted DC=0 for Christchurch
Successfully deleted DC=0 for Darfield
Successfully deleted DC=0 for Denali
Successfully deleted DC=0 for Duzce
Successfully deleted DC=0 for Emilia
Successfully deleted DC=0 for Haiti
Successfully deleted DC=0 for HectorMine
Successfully deleted DC=0 for Hokkaido
Successfully deleted DC=0 for Honduras
Successfully deleted DC=0 for Illapel
Successfully deleted DC=0 for Iquique
Successfully deleted DC=0 for Iwate
Successfully deleted DC=0 for Kobe
Successfully deleted DC=0 for Kocaeli
Successfully deleted DC=0 for Kumamoto
Successfully deleted DC=0 for LomaPrieta
Successfully deleted DC=0 for Maule
Successfully deleted DC=0 for Meinong
Successfully deleted DC=0 for MiyagiKen
Successfully deleted DC=0 for Muisne
Successfully deleted DC=0 for Napa
Successfully deleted DC=0 for Nepal
Successfully deleted DC=0 for Nigata1964
Successfully deleted DC=0 for Nigata2004
Successfully deleted DC=0 for Nigata2007
Su

In [37]:
ct = datetime.datetime.now() 
print("current time:-", ct)

current time:- 2021-03-14 18:33:59.837292


In [39]:
for x in range(7,num_eq):
    eq_name = eq_list['NewFolder'][x]
    region = eq_list['Region'][x]
    if eq_list['Working'][x]==1:
        
        CTI_proj = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_CTI_clipped_proj.tif"
        CTI = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_CTI_clipped.tif"
        spatial_ref = 4326
        
        arcpy.management.ProjectRaster(CTI_proj, CTI, spatial_ref, "BILINEAR")
        
        print("Successfully projected CTI for " + eq_name, str(x))

ct = datetime.datetime.now() 
print("current time:-", ct)

Successfully projected CTI for Chiba 7
Successfully projected CTI for ChiChi 8
Successfully projected CTI for ChinoHills 9
Successfully projected CTI for Christchurch 10
Successfully projected CTI for Darfield 11
Successfully projected CTI for Denali 12
Successfully projected CTI for Duzce 13
Successfully projected CTI for Emilia 14
Successfully projected CTI for Haiti 15
Successfully projected CTI for HectorMine 16
Successfully projected CTI for Hokkaido 17
Successfully projected CTI for Honduras 18
Successfully projected CTI for Illapel 19
Successfully projected CTI for Iquique 20
Successfully projected CTI for Kobe 22
Successfully projected CTI for Kocaeli 23
Successfully projected CTI for LomaPrieta 25
Successfully projected CTI for Maule 26
Successfully projected CTI for Meinong 27
Successfully projected CTI for MiyagiKen 28
Successfully projected CTI for Muisne 29
Successfully projected CTI for Napa 30
Successfully projected CTI for Nepal 31
Successfully projected CTI for Nigata1

In [44]:
for x in range(7,num_eq):
    eq_name = eq_list['NewFolder'][x]
    region = eq_list['Region'][x]
    if eq_list['Working'][x]==1:
        
        points_copy = path + eq_name + "\\fish_non_liq_point_copy.shp"
        
        dr = region_path + "ASTER_DRs\\"+ region + "_river.tif"
        dl = region_path + "ASTER_DLs\\"+ region + "_lake.tif"   
        wbd_elev = region_path + "ASTER_DEMs\\" + region + "_wbd_elev.tif"
        AI = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\AI\\ai_et0.tif" 
        CTI = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_CTI_clipped.tif"
        elev = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_GMTED_clipped.tif"
        elev_std = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_GMTED_std_clipped.tif"
        precip = "R:\\Liquefaction_RemoteSensing\Projects\\GeospatialModel\\global_data_2020\\Precipitation\\wc2.1_prec_annual.tif"
        slope = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_slope.tif"
        soil_thickness = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\Soils\\Global_Soil_Regolith_Sediment_1304\\data\\average_soil_and_sedimentary-deposit_thickness.tif"
        TPI = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_TPI.tif"
        TRI = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\local_data_2021_Alex\\" + eq_name + "\\" + eq_name + "_TRI.tif" 
        upland_lowland = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\Soils\\Global_Soil_Regolith_Sediment_1304\\data\\land_cover_mask.tif"
        vs30 = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\vs30\\global_vs30.grd"
        wtd = "R:\\Liquefaction_RemoteSensing\\Projects\\LossEstimation\\FindingLSE\\GlobalDatasets\\global_wtd_fil1.tif"
        PGA = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\PGA_PGV\\" + eq_name + "\\pga_mean.flt"
        PGA_std = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\PGA_PGV\\" + eq_name + "\\pga_std.flt"
        PGV = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\PGA_PGV\\" + eq_name + "\\pgv_mean.flt"
        PGV_std = "R:\\Liquefaction_RemoteSensing\\Projects\\GeospatialModel\\global_data_2020\\PGA_PGV\\" + eq_name + "\\pgv_std.flt"

        # DC should be first but ignoring because already grabbed it in the last section   
        ExtractMultiValuesToPoints(points_copy, [[dr,"DR"],[dl,"DL"],[wbd_elev,"WBD_Elev"],[AI,"AI"],[CTI,"CTI"],[elev,"Elev"],
                                            [elev_std,"Elev_std"],[precip,"Precip"],[slope,"Slope"],[soil_thickness,"Soil_thick"],
                                            [TPI,"TPI"],[TRI,"TRI"],[upland_lowland,"Upland_low"],[vs30,"Vs30"],[wtd,"WatTabDep"],
                                            [PGA,"PGA"],[PGA_std,"PGA_std"],[PGV,"PGV"],[PGV_std,"PGV_std"]],"NONE")
      
    else:
        pass
    
    print("Successfully extracted for " + str(x), eq_name)
    ct = datetime.datetime.now() 
    print("current time:-", ct)
    


Successfully extracted for 7 Chiba
current time:- 2021-03-14 20:46:27.534072
Successfully extracted for 8 ChiChi
current time:- 2021-03-14 20:46:54.275743
Successfully extracted for 9 ChinoHills
current time:- 2021-03-14 20:47:16.509878
Successfully extracted for 10 Christchurch
current time:- 2021-03-14 20:47:36.401205
Successfully extracted for 11 Darfield
current time:- 2021-03-14 20:48:01.115957
Successfully extracted for 12 Denali
current time:- 2021-03-14 20:49:40.071490
Successfully extracted for 13 Duzce
current time:- 2021-03-14 20:50:02.258455
Successfully extracted for 14 Emilia
current time:- 2021-03-14 20:50:24.081029
Successfully extracted for 15 Haiti
current time:- 2021-03-14 20:50:47.008318
Successfully extracted for 16 HectorMine
current time:- 2021-03-14 20:51:12.557580
Successfully extracted for 17 Hokkaido
current time:- 2021-03-14 20:52:10.981884
Successfully extracted for 18 Honduras
current time:- 2021-03-14 20:52:34.005048
Successfully extracted for 19 Illapel
